In [1]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r"THE_PATH_TO_YOUR_CONFIG_JSON_FROM_GOOGLE"
print(os.path.exists(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

True


In [ ]:
!pip install google-cloud-texttospeech python-slugify pandas

In [ ]:
import os
import re
import pandas as pd
from slugify import slugify
from google.cloud import texttospeech
from google.api_core.exceptions import InvalidArgument

CSV_PATH   = "EN.csv"  # the path to csv you want to voice
COLUMN     = "head_ru"  # specify the column name
OUT_DIR    = "tts_ru_zephyr_long_only" # output directory
PACE       = 0.95
PITCH      = 0.0
USE_EU_EP  = False
START      = 0
LIMIT      = None

os.makedirs(OUT_DIR, exist_ok=True)

if USE_EU_EP:
    from google.api_core.client_options import ClientOptions
    client = texttospeech.TextToSpeechClient(
        client_options=ClientOptions(api_endpoint="eu-texttospeech.googleapis.com")
    )
else:
    client = texttospeech.TextToSpeechClient()

LANG_CODE      = "ru-RU" # en-GB, de-DE 
VOICE_PRIMARY  = "ru-RU-Chirp3-HD-Zephyr" 
VOICE_FALLBACK = "ru-RU-Chirp-HD-Zephyr"

voices = client.list_voices().voices
def has_voice(name: str) -> bool:
    return any(v.name == name for v in voices)

if not has_voice(VOICE_PRIMARY):
    print(f"Warning: {VOICE_PRIMARY} not found in list_voices(). Will still try it.")
if not has_voice(VOICE_FALLBACK):
    print(f"Warning: {VOICE_FALLBACK} not found in list_voices(). Fallback may fail.")


def expand_slashes_en(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    # word1 / word2 -> word1 or word2
    text = re.sub(r"\b(\w+)\s*/\s*(\w+)\b", r"\1 или \2", text) # replace или with or/oder (depends on a language obviously)
    return text


def clean_idiom(raw: str) -> str:
    if not isinstance(raw, str):
        raw = str(raw)
    s = raw.strip()
    s = s.replace("—", "-").replace("–", "-")
    s = re.sub(r"\s+", " ", s)
    s = s.lower()
    return s

def build_long_context_en(idiom_tts: str) -> str:
    # Change the language of the text if needed
    return (
    f"В этом примере вы услышите русскую идиому: «{idiom_tts}». "
    f"Эта идиома часто используется в повседневной речи и имеет переносное или образное значение. "
    f"Я повторю её ещё раз, чтобы произношение было хорошо слышно: {idiom_tts}. "
    f"И напоследок вы услышите эту идиому в третий раз, в спокойном и естественном темпе речи: {idiom_tts}."
    )

def synthesize_en(text: str, voice_name: str):
    synthesis_input = texttospeech.SynthesisInput(text=text)
    voice = texttospeech.VoiceSelectionParams(
        language_code=LANG_CODE,
        name=voice_name,
    )
    audio_config = texttospeech.AudioConfig(
        audio_encoding=texttospeech.AudioEncoding.MP3,
        speaking_rate=PACE,
        pitch=PITCH,
    )
    return client.synthesize_speech(
        input=synthesis_input,
        voice=voice,
        audio_config=audio_config,
    )

df = pd.read_csv(CSV_PATH, sep=";")
if COLUMN not in df.columns:
    raise ValueError(f"Column '{COLUMN}' not found. Available: {list(df.columns)}")

series = df[COLUMN]


if LIMIT is not None:
    series = series.iloc[START:START+LIMIT]
else:
    series = series.iloc[START:]


if "id" in df.columns:
    ids = df.loc[series.index, "id"].astype(int).tolist()
else:
    ids = list(range(1 + START, 1 + START + len(series)))

ok, fail = 0, 0

for (row_idx, idiom_raw), num in zip(series.items(), ids):
    # skip NaNs
    if pd.isna(idiom_raw):
        print(f"skip (NaN) id={num}")
        continue

    raw = str(idiom_raw).strip()
    if not raw:
        print(f"skip (empty) id={num}")
        continue

    
    idiom_norm = clean_idiom(raw)

    idiom_tts = expand_slashes_en(idiom_norm)
    text      = build_long_context_en(idiom_tts)

    base = f"{int(num):04d}_{slugify(idiom_norm)[:60]}"
    out_path = os.path.join(OUT_DIR, f"{base}.mp3")

    if os.path.exists(out_path):
        print("skip (exists)", out_path)
        continue

    try:
        resp = synthesize_en(text, VOICE_PRIMARY)
    except InvalidArgument as e:
        msg = str(e).lower()
        if "requires a model" in msg or "invalid voice" in msg:
            try:
                resp = synthesize_en(text, VOICE_FALLBACK)
            except Exception as e2:
                print(f"{num}: {raw} → Chirp fallback failed:", e2)
                fail += 1
                continue
        else:
            print(f"{num}: {raw} →", e)
            fail += 1
            continue
    except Exception as e:
        print(f"{num}: {raw} →", e)
        fail += 1
        continue

    with open(out_path, "wb") as f:
        f.write(resp.audio_content)
    print("Done!", out_path)
    ok += 1

print(f"\nDone. Saved: {ok}, Failed: {fail}")
print("Output folder:", os.path.abspath(OUT_DIR))


## Extracting actual idioms with Whisper

In [ ]:
!pip install -U openai-whisper

In [ ]:
import os
import re
import math
import pandas as pd
from slugify import slugify
from pydub import AudioSegment
import whisper
from pydub import silence
from pydub.silence import detect_silence

In [ ]:
import os
import re
import pandas as pd
from slugify import slugify
from pydub import AudioSegment
from pydub.silence import detect_silence
import whisper

CSV_PATH    = "/content/drive/My Drive/Dubrovin/CSVS/Final/EN.csv" # your  CSV
COLUMN      = "head_ru"                                            # idiom column
AUDIO_DIR   = "/content/drive/My Drive/Dubrovin/tts_ru_zephyr_long_only" # long context audios
OUTPUT_DIR  = "/content/drive/My Drive/Dubrovin/tts_ru_SHORT_FIXED"  # where to save idiom-only
LANG_CODE   = "ru"
MODEL_NAME  = "small"
MIN_MATCH_RATIO = 0.6  # how strict matching idiom words vs ASR


os.makedirs(OUTPUT_DIR, exist_ok=True)

print("🔊 Loading Whisper model:", MODEL_NAME)
model = whisper.load_model(MODEL_NAME)


def normalize_token(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"[^\wäöüß]+", "", s, flags=re.UNICODE)  # keep letters/digits/umlauts
    return s

def tokenize_idiom(idiom: str):
    return [normalize_token(t) for t in re.split(r"\s+", idiom) if normalize_token(t)]

def flatten_whisper_words(result):
    words = []
    for seg in result.get("segments", []):
        for w in seg.get("words", []):
            text = w.get("word", "")  # often with leading space
            words.append({
                "text": text,
                "norm": normalize_token(text),
                "start": float(w.get("start", 0.0)),
                "end": float(w.get("end", 0.0)),
            })
    return words

import math

def find_loose_idiom_span(words, idiom_tokens, min_fraction=0.5):
    """
    Fallback: instead of requiring a contiguous window,
    we find all words whose norm is in idiom_tokens and
    take min(start), max(end) over them.

    min_fraction = fraction of idiom tokens that must be present
    to accept the span (e.g. 0.5 = at least half the words).
    """
    if not words or not idiom_tokens:
        return None, None

    # map token -> list of (start, end) times
    matches = []
    for w in words:
        if w["norm"] in idiom_tokens:
            matches.append((w["start"], w["end"]))

    if not matches:
        return None, None

    needed = max(1, math.ceil(len(idiom_tokens) * min_fraction))
    # how many distinct idiom tokens we saw at least once
    seen_tokens = set()
    for w in words:
        if w["norm"] in idiom_tokens:
            seen_tokens.add(w["norm"])

    if len(seen_tokens) < needed:
        return None, None

    # span from first to last matched word
    starts = [s for (s, e) in matches]
    ends   = [e for (s, e) in matches]
    return min(starts), max(ends)


def find_best_idiom_span(words, idiom_tokens, min_ratio=0.6):
    if not words or not idiom_tokens:
        return None, None

    n = len(words)
    m = len(idiom_tokens)
    best_score = 0.0
    best_span = (None, None)

    for i in range(0, max(0, n - m + 1)):
        window = words[i:i+m]
        matches = 0
        for j, tok in enumerate(idiom_tokens):
            if not tok:
                continue
            if window[j]["norm"] == tok:
                matches += 1
        ratio = matches / m
        if ratio > best_score:
            best_score = ratio
            start_sec = window[0]["start"]
            end_sec   = window[-1]["end"]
            best_span = (start_sec, end_sec)

    if best_score >= min_ratio and best_span[0] is not None and best_span[1] > best_span[0]:
        return best_span
    else:
        return None, None

def strip_silence_segment(segment, silence_db_offset=30, padding_ms=80):
    if len(segment) == 0:
        return segment

    silence_thresh = segment.dBFS - silence_db_offset

    silences = detect_silence(
        segment,
        min_silence_len=150,    # ms
        silence_thresh=silence_thresh
    )

    if not silences:
        return segment  # nothing detected

    leading_trim = 0
    trailing_trim = len(segment)

    for start, end in silences:
        if start == 0:
            # silence at start
            leading_trim = max(leading_trim, end)
        if end == len(segment):
            # silence at end
            trailing_trim = min(trailing_trim, start)

    if trailing_trim <= leading_trim:
        return segment

    start_ms = max(0, leading_trim - padding_ms)
    end_ms   = min(len(segment), trailing_trim + padding_ms)
    return segment[start_ms:end_ms]

def cut_audio_segment(input_path, start_sec, end_sec, output_path):
    audio = AudioSegment.from_file(input_path)
    start_ms = int(start_sec * 1000)
    end_ms   = int(end_sec * 1000)

    start_ms = max(0, min(start_ms, len(audio)))
    end_ms   = max(0, min(end_ms, len(audio)))
    if end_ms <= start_ms:
        raise ValueError("end_ms <= start_ms after clamping")

    segment = audio[start_ms:end_ms]

    # trim silence at edges
    segment = strip_silence_segment(
        segment,
        silence_db_offset=30,
        padding_ms=80          # leave 80ms at both ends
    )


    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    segment.export(output_path, format="mp3")


files = [f for f in os.listdir(AUDIO_DIR) if f.lower().endswith(".mp3")]
prefix_map = {}
for f in files:
    prefix = f.split("_", 1)[0]
    prefix_map.setdefault(prefix, []).append(f)

print(f" Found {len(files)} long audio files in {AUDIO_DIR}")

df = pd.read_csv(CSV_PATH, sep=";")
if COLUMN not in df.columns:
    raise ValueError(f"Column '{COLUMN}' not in CSV. Available: {list(df.columns)}")

if "id" in df.columns:
    ids = df["id"].astype(int).tolist()
else:
    ids = list(range(1, len(df) + 1))



In [ ]:
ok, fail, skipped = 0, 0, 0

for idx, row in df.iterrows():
    num = ids[idx]
    raw = row[COLUMN]

    if pd.isna(raw) or str(raw).strip() == "":
        print(f" skip (empty/Nan) id={num}")
        skipped += 1
        continue

    idiom_raw = str(raw).strip()
    idiom_tokens = tokenize_idiom(idiom_raw)
    if not idiom_tokens:
        print(f" skip (no tokens) id={num} text={idiom_raw!r}")
        skipped += 1
        continue

    prefix = f"{int(num):04d}"
    candidates = prefix_map.get(prefix, [])
    if not candidates:
        print(f"! id={num} ({idiom_raw!r}) → no audio file starting with {prefix}_* in {AUDIO_DIR}")
        fail += 1
        continue


    audio_file = candidates[0]
    audio_path = os.path.join(AUDIO_DIR, audio_file)

    base_slug = slugify(idiom_raw.lower())[:60]
    out_file  = f"{prefix}_{base_slug}_idiom.mp3"
    out_path  = os.path.join(OUTPUT_DIR, out_file)

    if os.path.exists(out_path):
        print(f" skip (exists) {out_path}")
        skipped += 1
        continue

    print(f"\n id={num} | file={audio_file} | idiom={idiom_raw}")

    # Transcribe with word timestamps
    try:
        result = model.transcribe(
            audio_path,
            language=LANG_CODE,
            word_timestamps=True,
            verbose=False
        )
    except Exception as e:
        print(f" id={num} → Whisper error:", e)
        fail += 1
        continue

    words = flatten_whisper_words(result)
    if not words:
        print(f" id={num} → no word-level timestamps from Whisper")
        fail += 1
        continue

    start_sec, end_sec = find_best_idiom_span(words, idiom_tokens, MIN_MATCH_RATIO)
    if start_sec is None or end_sec is None:
        print(f" id={num} → could not align idiom in ASR transcript (best match < {MIN_MATCH_RATIO})")
        fail += 1
        continue

    # small padding
    start_pad = 0.1
    end_pad   = 0.3
    start_sec = max(0.0, start_sec - start_pad)
    end_sec   = end_sec + end_pad

    try:
        cut_audio_segment(audio_path, start_sec, end_sec, out_path)
        print(f" saved idiom slice → {out_path}")
        ok += 1
    except Exception as e:
        print(f" id={num} → error cutting audio:", e)
        fail += 1

print("\n=== DONE ===")
print(" Slices created:", ok)
print(" Skipped:", skipped)
print(" Failed:", fail)
print("Output folder:", os.path.abspath(OUTPUT_DIR))

In [ ]:
# 2nd pass only processes failed (missing) idioms, same logic

extra_ok, extra_fail, extra_skipped = 0, 0, 0

for idx, row in df.iterrows():
    num = ids[idx]
    raw = row[COLUMN]

    if pd.isna(raw) or str(raw).strip() == "":
        extra_skipped += 1
        continue

    idiom_raw = str(raw).strip()
    prefix = f"{int(num):04d}"
    base_slug = slugify(idiom_raw.lower())[:60]
    out_file  = f"{prefix}_{base_slug}_idiom.mp3"
    out_path  = os.path.join(OUTPUT_DIR, out_file)

    # if file already exists, skip
    if os.path.exists(out_path):
        extra_skipped += 1
        continue

    candidates = prefix_map.get(prefix, [])
    if not candidates:
        print(f"[2nd pass] id={num} ({idiom_raw!r}) → no long audio")
        extra_fail += 1
        continue

    audio_file = candidates[0]
    audio_path = os.path.join(AUDIO_DIR, audio_file)

    print(f"\n2nd pass for id={num} | file={audio_file} | idiom={idiom_raw}")

    idiom_tokens = tokenize_idiom(idiom_raw)
    if not idiom_tokens:
        print(f"[2nd pass] no tokens for id={num}")
        extra_skipped += 1
        continue

    try:
        result = model.transcribe(
            audio_path,
            language=LANG_CODE,
            word_timestamps=True,
            verbose=False
        )
    except Exception as e:
        print(f"[2nd pass] id={num} → Whisper error:", e)
        extra_fail += 1
        continue

    words = flatten_whisper_words(result)
    if not words:
        print(f"[2nd pass] id={num} → no word timestamps")
        extra_fail += 1
        continue

    start_sec, end_sec = find_best_idiom_span(words, idiom_tokens, min_ratio=0.4)
    if start_sec is None or end_sec is None:
        print(f"[2nd pass] id={num} → still no good match")
        extra_fail += 1
        continue

    start_pad = 0.1
    end_pad   = 0.3
    start_sec = max(0.0, start_sec - start_pad)
    end_sec   = end_sec + end_pad

    try:
        cut_audio_segment(audio_path, start_sec, end_sec, out_path)
        print(f"[2nd pass] saved idiom slice → {out_path}")
        extra_ok += 1
    except Exception as e:
        print(f"[2nd pass] id={num} → error cutting audio:", e)
        extra_fail += 1

print("\n=== 2nd pass done ===")
print("Extra slices created:", extra_ok)
print("Extra skipped:", extra_skipped)
print("Extra failed:", extra_fail)
print("Output folder:", os.path.abspath(OUTPUT_DIR))